In [3]:
import pandas as pd

In [4]:
# Importing the sarcasm data textual representation

file_path = "sarcasm_data.json"

# Loading JSON file into a DataFrame

df_text = pd.read_json(file_path)
df_text = df_text.transpose()

In [5]:
df_text.head()

,utterance,speaker,context,context_speakers,show,sarcasm
160,It's just a privilege to watch your mind at work.,SHELDON,[I never would have identified the fingerprint...,"[LEONARD, SHELDON]",BBT,True
170,I don't think I'll be able to stop thinking ab...,PENNY,[This is one of my favorite places to kick bac...,"[HOWARD, PENNY, HOWARD, HOWARD, HOWARD, PENNY,...",BBT,True
180,"Since it's not bee season, you can have my epi...",SHELDON,"[Here we go. Pad thai, no peanuts., But does i...","[LEONARD, HOWARD, LEONARD]",BBT,False
190,"Lois Lane is falling, accelerating at an initi...",SHELDON,[A marathon? How many Superman movies are ther...,"[PENNY, SHELDON, PENNY, SHELDON, SHELDON, PENN...",BBT,False
1105,I'm just inferring this is a couch because the...,SHELDON,"[Great Caesar's ghost, look at this place., So...","[SHELDON, LEONARD, SHELDON, SHELDON, SHELDON, ...",BBT,True


In [6]:
df_text.shape

(690, 6)

In [7]:
df_text['speaker'].unique()

array(['SHELDON', 'PENNY', 'HOWARD', 'LEONARD', 'RAJ', 'PERSON',
       'BERNADETTE', 'AMY', 'PERSON3', 'PERSON1', 'CHANDLER', 'ROSS',
       'MONICA', 'JOEY', 'RACHEL', 'PHOEBE', 'DOROTHY', 'ROSE',
       'MEMBER-GIRL', 'MODERATOR', 'MEMBER-BOY'], dtype=object)

In [8]:
df_text['speaker']

160      SHELDON
170        PENNY
180      SHELDON
190      SHELDON
1105     SHELDON
          ...   
2169    CHANDLER
2235    CHANDLER
234     CHANDLER
2608    CHANDLER
2524    CHANDLER
Name: speaker, Length: 690, dtype: object

***Comment:***
As for our project it is essential for us to be able to label the different extracts by gender, we will remove all entries spoken by an ambiguous gender (such as 'Person1' or 'Moderator'). We remarked that this will remove 57 utternaces, which we deem acceptable for the size of our dataset.  

In [9]:
df_filtered = df_text[df_text['speaker'] != 'PERSON']
df_filtered = df_filtered[df_filtered['speaker'] != 'PERSON1']
df_filtered = df_filtered[df_filtered['speaker'] != 'PERSON3']
df_filtered = df_filtered[df_filtered['speaker'] != 'MODERATOR']
df_filtered.shape

(633, 6)

In [10]:
# Creating a dictionnary to label the different speakers according to their gender

gender_mapping = {
    'SHELDON': 'M',
    'PENNY': 'F',
    'HOWARD': 'M',
    'LEONARD': 'M',
    'RAJ': 'M',
    'BERNADETTE': 'F',
    'AMY': 'F',
    'CHANDLER': 'M',
    'ROSS': 'M',
    'MONICA': 'F',
    'JOEY': 'M',
    'RACHEL': 'F',
    'PHOEBE': 'F',
    'DOROTHY': 'F',
    'ROSE': 'F',
    'MEMBER-GIRL': 'F',
    'MEMBER-BOY': 'M',
}

# Defining a function to be able to map the speakers to their corresponding gender

def map_gender(speaker):
    return gender_mapping.get(speaker, 'Unknown')

# Adding a new column 'gender' based on the column 'speaker'
df_filtered['gender'] = df_filtered['speaker'].apply(map_gender)


In [11]:
df_filtered.head()

,utterance,speaker,context,context_speakers,show,sarcasm,gender
160,It's just a privilege to watch your mind at work.,SHELDON,[I never would have identified the fingerprint...,"[LEONARD, SHELDON]",BBT,True,M
170,I don't think I'll be able to stop thinking ab...,PENNY,[This is one of my favorite places to kick bac...,"[HOWARD, PENNY, HOWARD, HOWARD, HOWARD, PENNY,...",BBT,True,F
180,"Since it's not bee season, you can have my epi...",SHELDON,"[Here we go. Pad thai, no peanuts., But does i...","[LEONARD, HOWARD, LEONARD]",BBT,False,M
190,"Lois Lane is falling, accelerating at an initi...",SHELDON,[A marathon? How many Superman movies are ther...,"[PENNY, SHELDON, PENNY, SHELDON, SHELDON, PENN...",BBT,False,M
1105,I'm just inferring this is a couch because the...,SHELDON,"[Great Caesar's ghost, look at this place., So...","[SHELDON, LEONARD, SHELDON, SHELDON, SHELDON, ...",BBT,True,M


In [12]:
# Save the filtered data to a CSV file
df_filtered.to_csv('sarcasm_data.csv', index=False) 

In [13]:
M_data = df_filtered[df_filtered['gender'] == 'M']
F_data = df_filtered[df_filtered['gender'] == 'F']
print(M_data.shape, F_data.shape)

(430, 7) (203, 7)


In this project we will build 3 different models:
- mixed model (trained on male + female)
- male model
- female model

In order for the models to be unbiased, the training datasets have to be balanced, i.e. there should be the same number of sarcastic and non-sarcastic utterances.

In addition, to not be biased towards one gender, the mixed dataset should also be balanced accoring to gender.

Let's check if this is the case.

In [28]:
# Checking if the datasets are balanced
print('Checking label balance within the different datasets:')
if df_text[df_text['sarcasm'] == True].shape == df_text[df_text['sarcasm'] == False].shape:
    print('The original dataset is balanced')

if df_filtered[df_filtered['sarcasm'] == True].shape == df_filtered[df_filtered['sarcasm'] == False].shape:
    print('The filtered dataset is balanced')
else:
    print('Sarcastic utterances:', df_filtered[df_filtered['sarcasm'] == True].shape[0])
    print('Non sarcastic utterances:', df_filtered[df_filtered['sarcasm'] == False].shape[0])

if M_data[M_data['sarcasm'] == True].shape == M_data[M_data['sarcasm'] == False].shape:
    print('The male dataset is balanced')
else:
    print('Male sarcastic utterances:', M_data[M_data['sarcasm'] == True].shape[0])
    print('Male non sarcastic utterances:', M_data[M_data['sarcasm'] == False].shape[0])

if F_data[F_data['sarcasm'] == True].shape == F_data[F_data['sarcasm'] == False].shape:
    print('The female dataset is balanced')
else:
    print('Female sarcastic utterances:', F_data[F_data['sarcasm'] == True].shape[0])
    print('Female non sarcastic utterances:', F_data[F_data['sarcasm'] == False].shape[0])

print("")

# Checking if the dataset is balanced by gender
if df_filtered[df_filtered['gender'] == 'M'].shape == df_filtered[df_filtered['gender'] == 'F'].shape:
    print('The original dataset is balanced by gender')
else:
    print('Checking gender balance of the filtered dataset:')
    print('Male utterances:', df_filtered[df_filtered['gender'] == 'M'].shape[0])
    print('Female utterances:', df_filtered[df_filtered['gender'] == 'F'].shape[0])

Checking label balance within the different datasets:
The original dataset is balanced
Sarcastic utterances: 326
Non sarcastic utterances: 307
Male sarcastic utterances: 224
Male non sarcastic utterances: 206
Female sarcastic utterances: 102
Female non sarcastic utterances: 101

Checking gender balance of the filtered dataset:
Male utterances: 430
Female utterances: 203


As we can see the mixed dataset is not at all balanced by gender, as there are twice as many male utterances as female utterances. It is also not quite balanced in terms of sarcasm labels.

The male dataset is also not quite balanced, as there are a few more sarcastic utterances than non-sarcastic utterances.

The female dataset can be considered balances, as the difference between sarcastic and non-sarcastic utterances is only 1.

To balance the datasets we will use resampling and data augmentation.